Visualização dos componentes do PIB sob as óticas de Oferta e Demanda.


In [ ]:
import statsmodels.api as sm
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import requests
# hide all warnings
import warnings
import sys

sys.path.append('../')

if True:
    from _plots.figure import make_fig

warnings.filterwarnings('ignore')

A Macroeconomia é uma ciência que visa estudar o comportamento da economia como um todo. Para isso, ela busca explicar o comportamento de variáveis agregadas, tanto no curto quanto no longo prazo. Uma variável agregada é aquela que representa o comportamento de um conjunto de variáveis. Por exemplo, o PIB é uma variável agregada que representa o valor total de todos os bens e serviços produzidos em uma economia.

Nesta introdução, vamos nos concentrar na composição do PIB, com ênfase no Brasil. Ao entender os componentes do PIB, podemos obter insights sobre a saúde e o desempenho geral da economia. Para isso, vamos obter os dados do PIB e realizar o tratamento dos dados, que será o foco da primeira seção deste artigo.

Em seguida, na segunda seção, vamos analisar o PIB sob as duas óticas: Oferta e Demanda. Este estudo nos permitirá entender melhor como diferentes fatores e políticas econômicas podem impactar o PIB.

Por fim, na terceira seção, vamos concluir o artigo e listar as referências utilizadas. Esta seção servirá como um resumo dos pontos principais discutidos no artigo e fornecerá links para fontes adicionais de informação.

## Tratamento dos Dados

Nesta seção, antes da análise dos dados, vamos realizar o tratamento dos dados.


In [ ]:
#| code-fold: true
#| code-summary: Código da Aquisição dos dados
#| echo: true
#| label: code-get-data

# Endpoint para consulta dos dados do PIB
endpoint = "https://servicodados.ibge.gov.br/api/v3/agregados/1620/periodos/199601%7C199602%7C199603%7C199604%7C199701%7C199702%7C199703%7C199704%7C199801%7C199802%7C199803%7C199804%7C199901%7C199902%7C199903%7C199904%7C200001%7C200002%7C200003%7C200004%7C200101%7C200102%7C200103%7C200104%7C200201%7C200202%7C200203%7C200204%7C200301%7C200302%7C200303%7C200304%7C200401%7C200402%7C200403%7C200404%7C200501%7C200502%7C200503%7C200504%7C200601%7C200602%7C200603%7C200604%7C200701%7C200702%7C200703%7C200704%7C200801%7C200802%7C200803%7C200804%7C200901%7C200902%7C200903%7C200904%7C201001%7C201002%7C201003%7C201004%7C201101%7C201102%7C201103%7C201104%7C201201%7C201202%7C201203%7C201204%7C201301%7C201302%7C201303%7C201304%7C201401%7C201402%7C201403%7C201404%7C201501%7C201502%7C201503%7C201504%7C201601%7C201602%7C201603%7C201604%7C201701%7C201702%7C201703%7C201704%7C201801%7C201802%7C201803%7C201804%7C201901%7C201902%7C201903%7C201904%7C202001%7C202002%7C202003%7C202004%7C202101%7C202102%7C202103%7C202104%7C202201%7C202202%7C202203%7C202204%7C202301%7C202302/variaveis/583?localidades=N1[all]&classificacao=11255[90687,90691,90696,90707,93404,93405,93406,93407,93408]"

r = requests.get(endpoint)
raw_data = r.json()

# Tratamento dos dados brutos para um formato tabular
table_data = []
for entry in raw_data:
    for result in entry["resultados"]:
        for serie_i in result["series"]:

            for i, date_and_value in enumerate(serie_i["serie"].items()):
                date, value = date_and_value
                formated_date = date[:4] + "-" + date[4:] + "-01"

                assert len(result['classificacoes']) == 1, \
                    "Mais de uma classificação, possível perda de dados"

                table_data.append({
                    "id": result['classificacoes'][0]['id'],
                    "nome": list(result['classificacoes'][0]['categoria'].values())[0],
                    "date": formated_date,
                    "value": value
                })


# Criação do DataFrame
df = pd.DataFrame(table_data)

datasets = {
    nome: df[df["nome"] == nome]
    for nome in df["nome"].unique()
}

# Mapeamento dos nomes
nomes_mapper = {
    'Agropecuária - total': 'Agropecuária',
    'Indústria - total': 'Indústria',
    'Serviços - total': 'Serviços',
    'PIB a preços de mercado': 'PIB',
    'Despesa de consumo das famílias': 'Consumo',
    'Despesa de consumo da administração pública': 'Governo',
    'Formação bruta de capital fixo': 'Investimento',
    'Exportação de bens e serviços': 'Exportação',
    'Importação de bens e serviços (-)': 'Importação'
}
datasets = {
    nomes_mapper[nome]: dataset
    for nome, dataset in datasets.items()
}

# Conversão de dados númericos e datas
for nome, dataset in datasets.items():
    # to numeric
    dataset["value"] = pd.to_numeric(dataset["value"])
    dataset.index = pd.to_datetime(dataset["date"])
    # Set PIB Freq
    dataset = dataset.asfreq('Q')

pib = datasets['PIB']['value']

# Seleção dos componentes da Oferta e Demanda
product_components = [
    "Consumo",
    "Investimento",
    "Governo",
    "Exportação",
    "Importação"
]

income_components = [
    "Agropecuária",
    "Indústria",
    "Serviços"
]

# Bases finais
product_dataset = pd.DataFrame({
    nome: dataset['value']
    for nome, dataset in datasets.items()
    if nome in product_components
})

income_dataset = pd.DataFrame({
    nome: dataset['value']
    for nome, dataset in datasets.items()
    if nome in income_components
})

Para entender melhor a evolução do PIB ao longo do tempo, é importante considerar os dados dessazonalizados, ou seja, remover os efeitos recorrentes de cada período. Isso nos permite visualizar melhor a tendência dos dados.

Existem duas principais formas que uma série temporal pode ser decomposta: Aditiva e Multiplicativa. Ambas são representadas pela equação @eq-decomposicao-sazonalidade. No nosso caso, vamos utilizar a decomposição aditiva para o PIB.

$$
    y_t = T_t + S_t + C_t + \epsilon_t = T_t \times S_t \times C_t \times \epsilon_t
$$ {#eq-decomposicao-sazonalidade}

Onde: $y_t$ é o valor observado, $T_t$ é a tendência, $S_t$ é a sazonalidade, $C_t$ é o ciclo e $\epsilon_t$ é o erro.

Para obter os dados dessazonalizados, vamos utilizar a função `seasonal_decompose` da biblioteca [statsmodels](https://www.statsmodels.org/stable/index.html). Essa função retorna um objeto com os 4 componentes da decomposição. Para obter apenas a tendência, vamos utilizar o componente `trend`.


In [ ]:
# | code-summary: "Função para desasonalizar os dados"
# | echo: true


def seasonal_decompose(dataset: pd.DataFrame, period: int = 4):
    """Dessazonaliza os dados

    Args:
        dataset (pd.DataFrame): DataFrame com os dados
        period (int, optional): Periodo da sazonalidade. 
                                Padrao: 4, para o trimestre.

    Returns:
        pd.DataFrame: DataFrame com os dados desasonalizados
    """

    return pd.DataFrame({
        column: sm.tsa.seasonal_decompose(
            dataset[column], model='additive', period=period).trend
        for column in dataset.columns
    })

In [ ]:
# | code-fold: true
# | code-summary: "Cálculo dos dados dessazonalizados"
# | echo: true


product_dataset_des = seasonal_decompose(
    product_dataset, period=4
)

income_dataset_des = seasonal_decompose(
    income_dataset, period=4
)

pib_des = seasonal_decompose(
    pib.to_frame(), period=4
).iloc[:, 0]

Com os dados dessazonalizados, podemos calcular a variação percentual de cada componente do PIB. Para isso, vamos utilizar a função `pct_change` da biblioteca [pandas](https://pandas.pydata.org/). Essa função retorna a variação percentual entre o valor atual e o valor anterior.

E para calcular a variação cumulativa, vamos utilizar a função `cumprod` da biblioteca [pandas](https://pandas.pydata.org/). Essa função retorna a variação cumulativa entre o valor atual e o valor anterior. Assim, estudar a varição cumulativa de cada componente do PIB para entender melhor a evolução do PIB ao longo do tempo.


In [ ]:
# | code-fold: true
# | code-summary: "Cáculo da evolução cumulativa dos componentes do PIB"
# | echo: true


# Variação cumulativa da Oferta
product_dataset_cumulative = pd.DataFrame({
    nome: (1 + product_dataset_des[nome].pct_change()).cumprod()
    for nome in product_dataset_des.columns
})


# Transformação das Exportações e Importações em Exportações Líquidas
product_dataset_cumulative['Exportação Líquida'] = (1 +
                                                    product_dataset_cumulative['Exportação'].pct_change() - product_dataset_cumulative['Importação'].pct_change()).cumprod()
product_dataset_cumulative = product_dataset_cumulative.drop(
    ['Importação', 'Exportação'], axis=1)

# Variação cumulativa da Renda
income_dataset_cumulative = pd.DataFrame({
    nome: (1 + income_dataset_des[nome].pct_change()).cumprod()
    for nome in income_dataset_des.columns
})

# Variação cumulativa do PIB
pib_cumulative = (1 + pib_des.pct_change()).cumprod()

## Como estimar o PIB?

Nesse estudo, vamos analisar o PIB sob as duas óticas: Oferta e Demanda. Para isso, utilizaremos os dados do Instituto Brasileiro de Geografia e Estatística (IBGE), que são disponibilizados por meio do banco de dados SIDRA [^1], que foram tratados na seção anterior.


In [ ]:
def add_recessions_to_fig(fig, low, top):

    high_shift = .23
    low_shift = .4

    annot_shift = .02

    fig.add_shape(
        # Vertical Line
        type="line",
        x0="2008-01-01",
        y0=low,
        x1="2008-01-01",
        y1=top-high_shift-annot_shift,
        line=dict(
            color="gray",
            width=1,
            dash="dot",
        )
    )

    fig.add_shape(
        # Vertical Line
        type="line",
        x0="2015-01-01",
        y0=low,
        x1="2015-01-01",
        y1=top-low_shift-annot_shift,
        line=dict(
            color="gray",
            width=1,
            dash="dot",
        )
    )

    fig.add_shape(
        # Vertical Line
        type="line",
        x0="2020-01-01",
        y0=low,
        x1="2020-01-01",
        y1=top-high_shift-annot_shift,
        line=dict(
            color="gray",
            width=1,
            dash="dot",
        )
    )

    fig.add_annotation(
        x="2008-01-01",
        y=top-high_shift,
        text="Crise Financeira",
        showarrow=False,
        yshift=10
    )

    fig.add_annotation(
        x="2015-01-01",
        y=top-low_shift,
        text="Crise Governo<br>Dilma",
        showarrow=False,
        yshift=10
    )

    fig.add_annotation(
        x="2020-01-01",
        y=top-high_shift,
        text="Crise Covid-19",
        showarrow=False,
        yshift=10
    )

    return fig


### Oferta

Pela ótica da oferta podemos expressar o PIB com base em quatro componentes: Consumo, Investimento, Gastos do Governo e Exportações Líquidas. A ideia por trás dessa abordagem é que o PIB é a soma de todos os bens e serviços produzidos em uma economia. E para produzir esses bens e serviços, é necessário consumir, investir, gastar e exportar --- todos termos ligados à oferta. 

Na equação @eq-producao_agregada, temos a representação matemática dessa abordagem.

$$
    PIB = C + I + G + (X - M)
$$ {#eq-producao_agregada}

Onde: $C$ é o consumo, $I$ é o investimento, $G$ são os gastos do governo e $X - M$ é a diferença entre exportações e importações, que vamos chamar de exportações líquidas.

Analisando o historico dessazonalizado dos componentes do PIB na @fig-oferta, podemos observar que Investimentos e Exportações Líquidas foram os itens mais voláteis desde os anos 2000, principalmente em momentos de crise. Por outro lado, o consumo e os gastos do governo são mais estáveis.


In [ ]:
#| label: fig-oferta
#| fig-cap: Evolução dos componentes de produção do PIB desde 1996
low = -0.3
top = 1.5

fig = make_fig()

for nome in product_dataset_cumulative.columns:
    fig.add_trace(go.Scatter(
        x=product_dataset_cumulative.index,
        y=product_dataset_cumulative[nome]-1,
        name=nome,
        mode='lines',
        line=dict(width=2.5),
    ))

fig.add_trace(go.Scatter(
    x=pib_cumulative.index,
    y=pib_cumulative-1,
    name='PIB',
    mode='lines',
    line=dict(width=6),
    # set color
    marker_color='black'
))

fig.update_layout(
    title='A influência das commodities na produção brasileira<br><b>Exportações e Investimentos como motores mais relevantes<br>nos últimos anos.<br><sup>Evolução dos componentes de produção do PIB desde 1996</sup>')

fig.update_yaxes(tickformat=".0%", title='Variação acumulada',
                 range=[low, top], nticks=4)

# Add 2008, 2015 and 2020 recessions

fig = add_recessions_to_fig(fig, low, top)

fig.show()

O início da série se dá em 1996, logo após a instauração do Plano Real, que foi pautado por um tripé macroeconômico de câmbio flutuante, metas de inflação e responsabilidade fiscal. Essas políticas foram fundamentais para a estabilização da economia brasileira, e para o crescimento econômico que se seguiu. E logo após a sua implementação, podemos observar um crescimento significativo das Exportações Líquidas, que foi impulsionado pela força do Real, incentivando as exportações.

Este cenário de crescimento e estabilidade se estendeu até a eleição do presidente Lula, em 2002. Visto que o câmbio era flutuante, o Real se desvalorizou, após a eleição do presidente Lula, pois a percepção de risco aumentou. Na época o Dólar chegou a bater R$ 3,98, que foi o maior valor nominal por muitos anos. 

Já com o Presidente Lula no Governo, o Brasil passou por um outro período de forte crescimento econômico. Este crescimento foi impulsionado por um aumento significativo dos preços das commodities [^2], que por sua vez, impulsionou --- novamente --- o indicador de Exportações Líquidas. Portanto, podemos ver que as políticas econômicas implementadas durante o Plano Real e o aumento dos preços das commodities durante o primeiro mandato do presidente Lula tiveram um impacto significativo no crescimento econômico do Brasil.

Em 2008, o Brasil foi atingido pela crise financeira global. No período pré-crise, podemos observar dois aspectos: uma queda significativa nas Exportações Líquidas após o *boom das commodities* e um aumento nos Investimentos. No auge da crise, o Banco Central realizou a manutenção da taxa de juros, indo na direção contrária dos incentivos de crédito, fiscais e de intervenções realizados pelo governo [^3]. Matendo uma postura mais *hawkish*, o Banco Central conseguiu evitar uma deterioração mais drástica das expectativas dos agentes do mercado, e nos anos seguintes as taxas de juros foram reduzidas, o que contribuiu para o aumento dos Investimentos.

Já na crise de 2015, o Brasil passou por um período de forte recessão. Nesse período, podemos observar uma queda significativa nos Investimentos e um aumento repentino nas Exportações Líquidas. Essa queda nos Investimentos foi causada pela incerteza política e econômica, que afetou negativamente as expectativas dos agentes. Já o aumento nas Exportações Líquidas foi causado pela forte valorização do Real, que aumentou os incentivos para exportar e reduziu os incentivos para importar.

### Renda

Pela ótica da renda, podemos expressar o PIB com base em três componentes: Agropecuária, Indústria e Serviços. A ideia por trás dessa abordagem é que o PIB é a soma de todos os rendimentos gerados na economia. E para gerar esses rendimentos, é necessário produzir bens e serviços --- todos termos ligados à renda.

Na equação @eq-renda_agregada, temos a representação matemática dessa abordagem.

$$
    PIB = A + I + S
$$ {#eq-renda_agregada}

Onde: $A$ é a agropecuária, $I$ é a indústria e $S$ são os serviços.

Analisando o histórico desde os anos 2000, podemos observar grandes tendências na @fig-renda. A principal é o crescimento da Agropecuária, em detrimento da Indústria. A tendência inversa de tais fatores é um reflexo da economia brasileira, que no geral exporta matéria prima e importa produtos industrializados, e recentemente, o que reforça isso é a baixa exportação líquida e baixa renda da indústria. Olhando para os Serviços, podemos observar um ameno crescimento ao longo dos anos.

Diferente da ótica da oferta, a ótica da renda não é tão volátil. Então mesmo durante as crises, podemos observar uma estabilidade de longo prazo nos componentes da renda.


In [ ]:
#| label: fig-renda
#| fig-cap: Evolução dos componentes de renda do PIB desde 1996

# PIB sob a ótica da renda

low = -0.3
top = 1.5

fig = make_fig()

for nome in income_dataset_cumulative.columns:
    fig.add_trace(go.Scatter(
        x=income_dataset_cumulative.index,
        y=income_dataset_cumulative[nome]-1,
        name=nome,
        mode='lines',
        line=dict(width=2.5),
    ))

fig.add_trace(go.Scatter(
    x=pib_cumulative.index,
    y=pib_cumulative-1,
    name='PIB',
    mode='lines',
    line=dict(width=6),
    # set color
    marker_color='black'
))

fig.update_layout(
    title='Brasil: O País do Agro; Incapacidade de Evolução Industría;<br> e Serviços em leve Expansão.<br><sup>Evolução dos componentes de renda do PIB desde 1996</sup>')

fig.update_yaxes(tickformat=".0%", title='Variação acumulada',
                 range=[low, top], nticks=4)

# Add 2008, 2015 and 2020 recessions

fig = add_recessions_to_fig(fig, low, top)

fig.show()

Na crise de 2015, podemos observar uma queda nos 3 componentes da renda. Essa queda foi causada pela forte recessão, que mesmo com a alta das exportações líquidas (@fig-oferta), não foi suficiente para compensar a queda do consumo, investimento e gastos do governo, resultando uma maior queda na renda nos 3 componentes.

## Conclusão

Nesta introdução, vimos como o PIB pode ser estimado sob duas óticas: Oferta e Demanda. A ótica da oferta é mais volátil, e nos permite observar grandes variacões em períodos de crise. Já a ótica da renda é mais estável, e nos permite observar grandes tendências de longo prazo.

# Referências	

-  Introduction to Macroeconomics, Intermediate Macroeconomics: [Lecture 1](https://fgeerolf.com/econ102/intro-macro.html)

- Análise Qualitativa dos Dados do PIB: [Rpubs](https://rpubs.com/julio_costa/PIB_SIDRA)


[^1]: O SIDRA é um sistema que permite a consulta de dados e indicadores do IBGE. Para acessar os dados do PIB, basta acessar o link [https://sidra.ibge.gov.br/tabela/1620](https://sidra.ibge.gov.br/tabela/1620).

[^2]: O crescimento ocorreu em grande parte devido à crescente demanda das economias emergentes, principalmente da China: [https://pt.wikipedia.org/wiki/Boom_das_commodities_na_d%C3%A9cada_de_2000](https://pt.wikipedia.org/wiki/Boom_das_commodities_na_d%C3%A9cada_de_2000)

[^3]: Crise e perspectivas para a economia brasileira - [DOI: 10.1590/s0103-40142017.31890013](https://www.scielo.br/j/ea/a/jNpn6wfChvNj659nr4LLtGD/)